In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread

In [3]:

# Loading the tokenizer and model from Hugging Face's model hub.
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype = torch.float16)

In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(i

In [5]:
import numpy as np

def quantize_4bit(array):
    # Normalize the array to the range [-1, 1]
    max_abs_val = np.max(np.abs(array))
    normalized = array / max_abs_val

    # Scale to the range [-8, 7], round to nearest integer, and clamp
    quantized = np.round(normalized * 7)
    quantized = np.clip(quantized, -8, 7).astype(np.int8)

    return quantized

# Example usage
array = np.array([-3.0, -1.5, 0.0, 1.5, 3.0])
quantized_array = quantize_4bit(array)
print(quantized_array)


[-7 -4  0  4  7]


In [7]:
inputs = tokenizer('''once upon a time''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'

In [9]:
!rm -rf tinyllama
!mkdir tinyllama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
import os.path
import numpy as np


In [11]:

for name, param in model.named_parameters():
    print(name, param.size())
    if not os.path.isfile(f'tinyllama/{name}_weights.npy'):
        weights = param.data.numpy()
        #weights = quantize_4bit(weights)

        np.save(f'tinyllama/{name}_weights.npy', weights)

    if hasattr(param, 'bias'):
        if not os.path.isfile(f'tinyllama/{name}_bias.npy'):
            bias = param.bias.data.numpy()
            #bias = quantize_4bit(bias)
            np.save(f'tinyllama/{name}_bias.npy', bias)
    params = None
    weights = None
    bias = None

model.embed_tokens.weight torch.Size([32000, 2048])
model.layers.0.self_attn.q_proj.weight torch.Size([2048, 2048])
model.layers.0.self_attn.k_proj.weight torch.Size([256, 2048])
model.layers.0.self_attn.v_proj.weight torch.Size([256, 2048])
model.layers.0.self_attn.o_proj.weight torch.Size([2048, 2048])
model.layers.0.mlp.gate_proj.weight torch.Size([5632, 2048])
model.layers.0.mlp.up_proj.weight torch.Size([5632, 2048])
model.layers.0.mlp.down_proj.weight torch.Size([2048, 5632])
model.layers.0.input_layernorm.weight torch.Size([2048])
model.layers.0.post_attention_layernorm.weight torch.Size([2048])
model.layers.1.self_attn.q_proj.weight torch.Size([2048, 2048])
model.layers.1.self_attn.k_proj.weight torch.Size([256, 2048])
model.layers.1.self_attn.v_proj.weight torch.Size([256, 2048])
model.layers.1.self_attn.o_proj.weight torch.Size([2048, 2048])
model.layers.1.mlp.gate_proj.weight torch.Size([5632, 2048])
model.layers.1.mlp.up_proj.weight torch.Size([5632, 2048])
model.layers.1.m

model.layers.16.mlp.up_proj.weight torch.Size([5632, 2048])
model.layers.16.mlp.down_proj.weight torch.Size([2048, 5632])
model.layers.16.input_layernorm.weight torch.Size([2048])
model.layers.16.post_attention_layernorm.weight torch.Size([2048])
model.layers.17.self_attn.q_proj.weight torch.Size([2048, 2048])
model.layers.17.self_attn.k_proj.weight torch.Size([256, 2048])
model.layers.17.self_attn.v_proj.weight torch.Size([256, 2048])
model.layers.17.self_attn.o_proj.weight torch.Size([2048, 2048])
model.layers.17.mlp.gate_proj.weight torch.Size([5632, 2048])
model.layers.17.mlp.up_proj.weight torch.Size([5632, 2048])
model.layers.17.mlp.down_proj.weight torch.Size([2048, 5632])
model.layers.17.input_layernorm.weight torch.Size([2048])
model.layers.17.post_attention_layernorm.weight torch.Size([2048])
model.layers.18.self_attn.q_proj.weight torch.Size([2048, 2048])
model.layers.18.self_attn.k_proj.weight torch.Size([256, 2048])
model.layers.18.self_attn.v_proj.weight torch.Size([256, 

In [12]:
config = {'vocab_size': 32000,
 'max_position_embeddings': 2048,
 'hidden_size': 2048,
 'intermediate_size': 5632,
 'num_hidden_layers': 22,
 'num_attention_heads': 32,
 'num_key_value_heads': 4,
 'hidden_act': 'silu',
 'initializer_range': 0.02,
 'rms_norm_eps': 1e-05,
 'pretraining_tp': 1,
 'use_cache': True,
 'rope_theta': 10000.0,
 'rope_scaling': None,
 'attention_bias': False,
 'return_dict': True,
 'output_hidden_states': False,
 'output_attentions': False,
 'torchscript': False,
 'torch_dtype': 'bfloat16',
 'use_bfloat16': False,
 'tf_legacy_loss': False,
 'pruned_heads': {},
 'tie_word_embeddings': False,
 'is_encoder_decoder': False,
 'is_decoder': False,
 'cross_attention_hidden_size': None,
 'add_cross_attention': False,
 'tie_encoder_decoder': False,
 'max_length': 20,
 'min_length': 0,
 'do_sample': False,
 'early_stopping': False,
 'num_beams': 1,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'typical_p': 1.0,
 'repetition_penalty': 1.0,
 'length_penalty': 1.0,
 'no_repeat_ngram_size': 0,
 'encoder_no_repeat_ngram_size': 0,
 'bad_words_ids': None,
 'num_return_sequences': 1,
 'chunk_size_feed_forward': 0,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': None,
 'forced_eos_token_id': None,
 'remove_invalid_values': False,
 'exponential_decay_length_penalty': None,
 'suppress_tokens': None,
 'begin_suppress_tokens': None,
 'architectures': ['LlamaForCausalLM'],
 'finetuning_task': None,
 'id2label': {0: 'LABEL_0', 1: 'LABEL_1'},
 'label2id': {'LABEL_0': 0, 'LABEL_1': 1},
 'tokenizer_class': None,
 'prefix': None,
 'bos_token_id': 1,
 'pad_token_id': None,
 'eos_token_id': 2,
 'sep_token_id': None,
 'decoder_start_token_id': None,
 'task_specific_params': None,
 'problem_type': None,
 '_name_or_path': 'TinyLlama/TinyLlama-1.1B-Chat-v1.0',
 'transformers_version': '4.35.2',
 'model_type': 'llama'}

In [13]:

from os import environ
environ['OMP_NUM_THREADS'] = '16'
import numpy as np

In [14]:
class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


my_dict = {'id': 1, 'website': 'bobbyhadz.com', 'topic': 'Python'}

new_dict = AttributeDict(my_dict)

print(new_dict.website)  # 👉️ bobbyhadz.com
print(new_dict.topic)  # 👉️ Python

new_dict.author = 'Borislav Hadzhiev'

# 👇️ {'id': 1, 'website': 'bobbyhadz.com', 'topic': 'Python', 'author': 'Borislav Hadzhiev'}
print(new_dict)

del new_dict.author

# 👇️ {'id': 1, 'website': 'bobbyhadz.com', 'topic': 'Python'}
print(new_dict)


bobbyhadz.com
Python
{'id': 1, 'website': 'bobbyhadz.com', 'topic': 'Python', 'author': 'Borislav Hadzhiev'}
{'id': 1, 'website': 'bobbyhadz.com', 'topic': 'Python'}


In [15]:
config1 = AttributeDict(config)

In [16]:
import torch
import numpy as np
zeros = [0] * config['hidden_size']
embedding = np.random.randn(config['vocab_size'], config['hidden_size'])
vocab_size = config['vocab_size']

In [17]:
embedding = np.load('tinyllama/model.embed_tokens.weight_weights.npy', mmap_mode='r')

In [18]:
class RotaryEmbedding():
    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None):
        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.base = base
        inv_freq = 1.0 / (self.base ** (np.arange(0, self.dim, 2) / self.dim))
        self.inv_freq = inv_freq


        # Build here to make `torch.jit.trace` work.
        self._set_cos_sin_cache(
            seq_len=max_position_embeddings
        )

    def _set_cos_sin_cache(self, seq_len):
        self.max_seq_len_cached = seq_len
        t = np.arange(self.max_seq_len_cached)

        freqs = np.outer(t, self.inv_freq)
        # Different from paper, but it uses a different permutation in order to obtain the same calculation
        emb = np.concatenate((freqs, freqs), axis=-1)
        self.cos_cached = np.cos(emb)
        self.sin_cached = np.sin(emb)


    def __call__(self, x, seq_len=None):
        # x: [bs, num_attention_heads, seq_len, head_size]
        if seq_len > self.max_seq_len_cached:
            self._set_cos_sin_cache(seq_len=seq_len)

        return (
            self.cos_cached[:seq_len],
            self.sin_cached[:seq_len],
        )

In [19]:
rot = RotaryEmbedding(10)

In [20]:


import numpy as np

def rotate_half_np(x):
    """
    Rotates half the hidden dims of the input, implemented in NumPy.
    Args:
        x (numpy.ndarray): Input array.
    Returns:
        numpy.ndarray: Rotated array.
    """
    # Split the array into two halves along the last dimension
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]

    # Reverse the second half and concatenate with the first half
    # NumPy's negative indexing reverses the array
    return np.concatenate((-x2, x1), axis=-1)

def apply_rotary_pos_emb(q, k, cos, sin, position_ids, unsqueeze_dim=1):
    cos = np.expand_dims( cos[position_ids], unsqueeze_dim)
    sin = np.expand_dims( sin[position_ids], unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half_np(q) * sin)
    k_embed = (k * cos) + (rotate_half_np(k) * sin)
    return q_embed, k_embed

import numpy as np
import math

def gelu_np(input):
    return 0.5 * input * (1.0 + np.tanh(math.sqrt(2.0 / math.pi) * (input + 0.044715 * np.power(input, 3.0))))

# Example usage
input_np = np.random.randn(10)

import numpy as np

def silu(x):
    return x * (1 / (1 + np.exp(-x)))




# Example activation function mapping (adjust as needed)
ACT2FN_np = {
    'relu': np.vectorize(lambda x: max(0, x)),
    # Add other activation functions as needed
    'gelu_new': gelu_np,
    'silu': silu
}


In [21]:
import numpy as np

def repeat_kv_np(hidden_states, n_rep):
    """
    Replicates the behavior of torch.repeat_interleave for a specific use-case.
    The hidden states go from (batch, num_key_value_heads, seqlen, head_dim)
    to (batch, num_attention_heads, seqlen, head_dim) in NumPy.
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape

    if n_rep == 1:
        return hidden_states

    # Expand and then repeat the array along the third axis
    hidden_states_expanded = np.expand_dims(hidden_states, axis=2)
    hidden_states_repeated = np.tile(hidden_states_expanded, (1, 1, n_rep, 1, 1))

    # Reshape the array to the desired shape
    return hidden_states_repeated.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)

# Example usage
hidden_states = np.random.randn(2, 3, 4, 5)  # Example input tensor with shape (batch, num_key_value_heads, slen, head_dim)
n_rep = 2  # Example repetition factor
output = repeat_kv_np(hidden_states, n_rep)
print(output.shape)  # The shape should be (2, 6, 4, 5) in this example


(2, 6, 4, 5)


In [22]:
import numpy as np

class Linear_np:
    def __init__(self, in_features, out_features, bias=True):
        # Initialize weights and bias
        self.weights = np.random.randn(out_features, in_features)
        self.bias = np.random.randn(out_features)


    def load(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def __call__(self, x):
        # Perform the linear operation (y = xA^T + b)
        if self.bias is not None:
            return np.dot(x, self.weights.T) + self.bias
        return np.dot(x, self.weights.T)

# Example usage
input_features = 5
output_features = 3
linear_layer = Linear_np(input_features, output_features)

# Example input (batch_size x input_features)
x = np.random.randn(2, input_features)

# Forward pass
output = linear_layer(x)
print(output)


[[-0.75198482 -0.42175198 -2.04647616]
 [-0.04557262  1.76708609 -1.98061853]]


In [23]:
class MLP_np():
    def __init__(self, config, layer_index):

        self.config = config
        self.layer_index = layer_index
        self.activation_fn = ACT2FN_np[config.hidden_act]


        self.intermediate_size = config.intermediate_size
        self.gate_proj = Linear_np(config.hidden_size, self.intermediate_size, bias=False)
        self.up_proj = Linear_np(config.hidden_size, self.intermediate_size, bias=False)
        self.down_proj = Linear_np(self.intermediate_size, config.hidden_size, bias=False)
        self.act_fn = ACT2FN_np[config.hidden_act]


        weights = np.load(f'tinyllama/model.layers.{layer_index}.mlp.gate_proj.weight_weights.npy', mmap_mode='r')
        self.gate_proj.load(weights, bias=None)

        weights = np.load(f'tinyllama/model.layers.{layer_index}.mlp.up_proj.weight_weights.npy', mmap_mode='r')
        self.up_proj.load(weights, bias=None)

        weights = np.load(f'tinyllama/model.layers.{layer_index}.mlp.down_proj.weight_weights.npy', mmap_mode='r')
        self.down_proj.load(weights, bias=None)





    def __call__(self, x) :
        return self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))

In [24]:
import numpy as np

# model.layers.0.input_layernorm.weight torch.Size([4096])
# model.layers.0.post_attention_layernorm.weight torch.Size([4096])
# model.norm.weight torch.Size([4096])
class RMSNorm:
    def __init__(self, hidden_size, layer_idx=-1, post=False, eps=1e-6):
        """
        MistralRMSNorm is equivalent to T5LayerNorm, implemented in NumPy.
        """
        self.weight = np.ones(hidden_size)
        self.variance_epsilon = eps

        if layer_idx == -1:

            wt = np.load(f'tinyllama/model.norm.weight_weights.npy', mmap_mode='r')
            self.weight = wt
        else:
            if post:
                wt = np.load(f'tinyllama/model.layers.{layer_idx}.post_attention_layernorm.weight_weights.npy', mmap_mode='r')
            else:
                wt = np.load(f'tinyllama/model.layers.{layer_idx}.input_layernorm.weight_weights.npy', mmap_mode='r')

            self.weight = wt

    def __call__(self, hidden_states):
        # Compute variance
        variance = np.mean(np.square(hidden_states), axis=-1, keepdims=True)

        # Normalize hidden states
        hidden_states_normalized = hidden_states * np.reciprocal(np.sqrt(variance + self.variance_epsilon))

        # Apply learned weights
        return self.weight * hidden_states_normalized


In [25]:
x = np.random.randn(1,1,2048)
m = RMSNorm(2048, layer_idx=0, post=True)
m(x)

array([[[ 0.01501403, -0.0527625 , -0.27735808, ...,  0.04456021,
          0.01843926,  0.12067095]]])

In [26]:
import numpy as np

def softmax(x, axis=-1):
    """Compute softmax values for each sets of scores in x along the specified axis."""
    e_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

# Example usage
attn_weights_np = np.random.randn(2, 4, 10, 10).astype(np.float32)  # Example attention weights
softmax_attn_weights_np = softmax(attn_weights_np, axis=-1)

# If you need to convert back to a specific dtype, like in PyTorch code, you can cast it:
# e.g., softmax_attn_weights_np = softmax_attn_weights_np.astype(original_dtype)


In [27]:
import numpy as np

# Original 2x2 matrix
matrix_2x2 = np.array([[1, 2],
                       [3, 4]])

# Repeat the matrix to get an 8x2x2 matrix
matrix_8x2x2 = np.repeat(matrix_2x2[np.newaxis, :, :], repeats=8, axis=0)

print(matrix_8x2x2)
print(matrix_8x2x2.shape)  # Should be (8, 2, 2)


[[[1 2]
  [3 4]]

 [[1 2]
  [3 4]]

 [[1 2]
  [3 4]]

 [[1 2]
  [3 4]]

 [[1 2]
  [3 4]]

 [[1 2]
  [3 4]]

 [[1 2]
  [3 4]]

 [[1 2]
  [3 4]]]
(8, 2, 2)


In [28]:
import numpy as np

def create_masked_context_mask(mask, diagonal=0, dtype=np.float32):
    """
    Apply a context mask to the given mask array.

    :param mask: The original mask array (2D or 4D).
    :param diagonal: The diagonal offset for the upper triangular part.
    :param dtype: The data type to determine the minimum value.
    :return: The masked array with applied context mask.
    """
    b,k, s, q = mask.shape
    # Create an upper triangular matrix (context mask)
    context_mask = 1 - np.tril(np.ones((s,q), dtype=int), k=0)

    context_mask = np.repeat(context_mask[np.newaxis, np.newaxis, :, :], repeats=k, axis=1)

  # Then, repeat the resulting matrix to get an (8, 32, 2, 2) matrix
    context_mask = np.repeat(context_mask, repeats=b, axis=0)


    # Apply the context mask to the original mask
    masked_mask = np.where(context_mask, np.finfo(dtype).min, mask)

    return masked_mask

# Example usage
mask = np.random.rand(1,1, 4, 5)  # Replace with your actual mask
diagonal = 0
masked_mask = create_masked_context_mask(mask, diagonal)
masked_mask

array([[[[ 2.48910188e-01, -3.40282347e+38, -3.40282347e+38,
          -3.40282347e+38, -3.40282347e+38],
         [ 9.38076493e-01,  4.77504169e-01, -3.40282347e+38,
          -3.40282347e+38, -3.40282347e+38],
         [ 3.19648611e-01,  2.80959636e-01,  8.94178996e-01,
          -3.40282347e+38, -3.40282347e+38],
         [ 9.23695647e-01,  9.14898012e-01,  8.64843013e-01,
           8.25265678e-01, -3.40282347e+38]]]])

In [29]:
class Attention():
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config, layer_idx = None):

        self.config = config
        self.layer_idx = layer_idx

        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.hidden_size // self.num_heads
        self.num_key_value_heads = config.num_key_value_heads
        self.num_key_value_groups = self.num_heads // self.num_key_value_heads
        self.max_position_embeddings = config.max_position_embeddings
        self.rope_theta = config.rope_theta
        self.is_causal = True

        if (self.head_dim * self.num_heads) != self.hidden_size:
            raise ValueError(
                f"hidden_size must be divisible by num_heads (got `hidden_size`: {self.hidden_size}"
                f" and `num_heads`: {self.num_heads})."
            )

        self.q_proj = Linear_np(self.hidden_size, self.num_heads * self.head_dim, bias=False)
        self.k_proj = Linear_np(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=False)
        self.v_proj = Linear_np(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=False)
        self.o_proj = Linear_np(self.num_heads * self.head_dim, self.hidden_size, bias=False)



        self._init_rope()


        weights =np.load(f'tinyllama/model.layers.{layer_idx}.self_attn.q_proj.weight_weights.npy', mmap_mode='r')
        self.q_proj.load(weights, bias=None)

        weights =np.load(f'tinyllama/model.layers.{layer_idx}.self_attn.k_proj.weight_weights.npy', mmap_mode='r')
        self.k_proj.load(weights, bias=None)

        weights =np.load(f'tinyllama/model.layers.{layer_idx}.self_attn.v_proj.weight_weights.npy', mmap_mode='r')
        self.v_proj.load(weights, bias=None)

        weights =np.load(f'tinyllama/model.layers.{layer_idx}.self_attn.o_proj.weight_weights.npy', mmap_mode='r')
        self.o_proj.load(weights, bias=None)



    def _init_rope(self):
        self.rotary_emb = RotaryEmbedding(
            self.head_dim,
            max_position_embeddings=self.max_position_embeddings,
            base=self.rope_theta,
        )


    def _split_heads(self, fused_qkv):
        batch_size, seq_length, _ = fused_qkv.shape
        reshaped = fused_qkv.reshape(batch_size, seq_length, self.num_heads, 3, self.head_dim)
        query = reshaped[..., 0, :]
        key = reshaped[..., 1, :]
        value = reshaped[..., 2, :]
        return query, key, value

    def __call__(
        self,
        hidden_states,
        attention_mask = None,
        position_ids = None,
        past_key_value = None,
        output_attentions = False,
        use_cache = False,
    ):
        bsz, q_len, _ = hidden_states.shape



        query_states = self.q_proj(hidden_states)
        key_states = self.k_proj(hidden_states)
        value_states = self.v_proj(hidden_states)



        query_states = query_states.reshape(bsz, q_len, self.num_heads, self.head_dim).transpose(0, 2, 1, 3)
        key_states = key_states.reshape(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(0, 2, 1, 3)
        value_states = value_states.reshape(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(0, 2, 1, 3)



        kv_seq_len = key_states.shape[-2]
        if past_key_value is not None:
            kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)
        cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)

        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)


        if past_key_value is not None:
            cache_kwargs = {"sin": sin, "cos": cos, "partial_rotation_size": self.rotary_emb.dim}
            key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)


        key_states = repeat_kv_np(key_states, self.num_key_value_groups)
        value_states = repeat_kv_np(value_states, self.num_key_value_groups)


        # Queries and keys upcast to fp32 is required by Phi-2 to avoid overflow
        attn_weights = np.matmul(
            query_states, key_states.transpose(0, 1, 3, 2)
        ) / np.sqrt(self.head_dim)

        if attention_mask is not None:
            if attention_mask.shape != (bsz, 1, q_len, kv_seq_len):
                print(bsz, 1, q_len, kv_seq_len)
                raise ValueError(
                    f"Attention mask should be of size {(bsz, 1, q_len, kv_seq_len)}, but is {attention_mask.size()}"
                )
            attn_weights = attn_weights + attention_mask

        # upcast attention to fp32



        attn_weights = create_masked_context_mask(attn_weights, 0)

        attn_weights = softmax(attn_weights, axis=-1)



        attn_output = np.matmul(attn_weights, value_states)



        attn_output = attn_output.transpose(0, 2, 1, 3)

        attn_output = attn_output.reshape(bsz, q_len, self.hidden_size)


        attn_output = self.o_proj(attn_output)

        return attn_output, attn_weights, past_key_value


In [30]:
attn = Attention(config1, 0)
x = np.random.randn(1,2,2048)
attn(x)[0].shape

(1, 2, 2048)

In [31]:
class DecoderLayer():
    def __init__(self, config, layer_idx: int):
        self.self_attn = Attention(config, layer_idx=layer_idx)
        self.mlp = MLP_np(config, layer_index=layer_idx)
        self.input_layernorm = RMSNorm(config.hidden_size, layer_idx=layer_idx, eps=config.rms_norm_eps)
        self.post_attention_layernorm = RMSNorm(config.hidden_size, layer_idx=layer_idx, post=True, eps=config.rms_norm_eps)

    def __call__(
        self,
        hidden_states,
        attention_mask = None,
        position_ids  = None,
        output_attentions  = False,
        use_cache  = False,
        past_key_value  = None,
    ) :
        """
        Args:
            hidden_states (`torch.FloatTensor`):
                input to the layer of shape `(batch, seq_len, embed_dim)`
            attention_mask (`torch.FloatTensor`, *optional*): attention mask of size
                `(batch, 1, tgt_len, src_len)` where padding elements are indicated by very large negative values.
            position_ids (`torch.LongTensor` of shape `({0})`, *optional*):
                Indices of positions of each input sequence tokens in the position embeddings. Selected in the range
                `[0, config.n_positions - 1]`. [What are position IDs?](../glossary#position-ids)
            output_attentions (`bool`, *optional*):
                Whether or not to return the attentions tensors of all attention layers. See `attentions` under
                returned tensors for more detail.
            use_cache (`bool`, *optional*):
                If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding
                (see `past_key_values`).
            past_key_value (`Tuple(torch.FloatTensor)`, *optional*): cached past key and value projection states
        """

        residual = hidden_states

        hidden_states = self.input_layernorm(hidden_states)

        # Self Attention
        attn_outputs, self_attn_weights, present_key_value = self.self_attn(
            hidden_states=hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_value=past_key_value,
            output_attentions=output_attentions,
            use_cache=use_cache,
        )
        attn_outputs = attn_outputs


        hidden_states = residual + attn_outputs

        residual = hidden_states

        hidden_states = self.post_attention_layernorm(hidden_states)
        hidden_states = self.mlp(hidden_states)
        hidden_states =  residual + hidden_states
        outputs = (hidden_states,)

        if output_attentions:
            outputs += (self_attn_weights,)

        if use_cache:
            outputs += (present_key_value,)

        return outputs

In [32]:
dec = DecoderLayer(config1, 0)
dec(x)

(array([[[-0.18924278, -1.1581644 ,  1.03201109, ...,  0.86278668,
          -1.7153241 , -1.87950691],
         [ 2.72335772,  0.93399983,  1.31827286, ...,  1.02743189,
           0.48744402,  1.10102729]]]),)

In [33]:
class Cache:
    """
    Base, abstract class for all caches. The actual data structure is specific to each subclass.
    """

    def update(
        self,
        key_states,
        value_states,
        layer_idx: int,
        cache_kwargs= None,
    ) :
        """
        Updates the cache with the new `key_states` and `value_states` for the layer `layer_idx`.

        Parameters:
            key_states (`torch.Tensor`):
                The new key states to cache.
            value_states (`torch.Tensor`):
                The new value states to cache.
            layer_idx (`int`):
                The index of the layer to cache the states for.
            cache_kwargs (`Dict[str, Any]`, `optional`):
                Additional arguments for the cache subclass. These are specific to each subclass and allow new types of
                cache to be created.

        Return:
            A tuple containing the updated key and value states.
        """
        raise NotImplementedError("Make sure to implement `update` in a subclass.")

    def get_seq_length(self, layer_idx= 0) :
        """Returns the sequence length of the cached states. A layer index can be optionally passed."""
        raise NotImplementedError("Make sure to implement `get_seq_length` in a subclass.")

    def get_max_length(self):
        """Returns the maximum sequence length of the cached states, if there is any."""
        raise NotImplementedError("Make sure to implement `get_max_length` in a subclass.")

    def get_usable_length(self, new_seq_length: int, layer_idx = 0):
        """Given the sequence length of the new inputs, returns the usable length of the cache."""
        # Cache without size limit -> all cache is usable
        # Cache with size limit -> if the length cache plus the length of the new inputs is larger the maximum cache
        #   length, we will need to evict part of the cache (and thus not all cache is usable)
        max_length = self.get_max_length()
        previous_seq_length = self.get_seq_length(layer_idx)
        if max_length is not None and previous_seq_length + new_seq_length > max_length:
            return max_length - new_seq_length
        return previous_seq_length

In [34]:
class DynamicCache(Cache):
    """
    A cache that grows dynamically as more tokens are generated. This is the default for generative models.

    It stores the Key and Value states as a list of tensors, one for each layer. The expected shape for each tensor is
    `[batch_size, num_heads, seq_len, head_dim]`.
    """

    def __init__(self) :
        self.key_cache = []
        self.value_cache= []
        self.seen_tokens = 0  # Used in `generate` to keep tally of how many tokens the cache has seen

    def __getitem__(self, layer_idx: int) :
        """
        Support for backwards-compatible `past_key_value` indexing, e.g. `past_key_value[0][0].shape[2]` to get the
        sequence length.
        """
        if layer_idx < len(self):
            return (self.key_cache[layer_idx], self.value_cache[layer_idx])
        else:
            raise KeyError(f"Cache only has {len(self)} layers, attempted to access layer with index {layer_idx}")

    def __iter__(self):
        """
        Support for backwards-compatible `past_key_value` iteration, e.g. `for x in past_key_value:` to iterate over
        keys and values
        """
        for layer_idx in range(len(self)):
            yield (self.key_cache[layer_idx], self.value_cache[layer_idx])

    def __len__(self):
        """
        Support for backwards-compatible `past_key_value` length, e.g. `len(past_key_value)`. This value corresponds
        to the number of layers in the model.
        """
        return len(self.key_cache)

    def update(
        self,
        key_states,
        value_states,
        layer_idx: int,
        cache_kwargs = None,
    ) :
        """
        Updates the cache with the new `key_states` and `value_states` for the layer `layer_idx`.

        Parameters:
            key_states (`torch.Tensor`):
                The new key states to cache.
            value_states (`torch.Tensor`):
                The new value states to cache.
            layer_idx (`int`):
                The index of the layer to cache the states for.
            cache_kwargs (`Dict[str, Any]`, `optional`):
                Additional arguments for the cache subclass. No additional arguments are used in `DynamicCache`.

        Return:
            A tuple containing the updated key and value states.
        """
        # Update the number of seen tokens
        if layer_idx == 0:
            self.seen_tokens += key_states.shape[-2]

        # Update the cache
        if len(self.key_cache) <= layer_idx:
            self.key_cache.append(key_states)
            self.value_cache.append(value_states)
        else:
            self.key_cache[layer_idx] = np.concatenate([self.key_cache[layer_idx], key_states], axis=-2)
            self.value_cache[layer_idx] = np.concatenate([self.value_cache[layer_idx], value_states], axis=-2)

        return self.key_cache[layer_idx], self.value_cache[layer_idx]

    def get_seq_length(self, layer_idx = 0) -> int:
        """Returns the sequence length of the cached states. A layer index can be optionally passed."""
        if len(self.key_cache) <= layer_idx:
            return 0
        return self.key_cache[layer_idx].shape[-2]

    def get_max_length(self) :
        """Returns the maximum sequence length of the cached states. DynamicCache does not have a maximum length."""
        return None

    def reorder_cache(self, beam_idx):
        """Reorders the cache for beam search, given the selected beam indices."""
        for layer_idx in range(len(self.key_cache)):
            device = self.key_cache[layer_idx].device
            self.key_cache[layer_idx] = self.key_cache[layer_idx].index_select(0, beam_idx.to(device))
            device = self.value_cache[layer_idx].device
            self.value_cache[layer_idx] = self.value_cache[layer_idx].index_select(0, beam_idx.to(device))

    def to_legacy_cache(self) :
        """Converts the `DynamicCache` instance into the its equivalent in the legacy cache format."""
        legacy_cache = ()
        for layer_idx in range(len(self)):
            legacy_cache += ((self.key_cache[layer_idx], self.value_cache[layer_idx]),)
        return legacy_cache

    @classmethod
    def from_legacy_cache(cls, past_key_values = None) :
        """Converts a cache in the legacy cache format into an equivalent `DynamicCache`."""
        cache = cls()
        if past_key_values is not None:
            for layer_idx in range(len(past_key_values)):
                key_states, value_states = past_key_values[layer_idx]
                cache.update(key_states, value_states, layer_idx)
        return cache


In [35]:
class Model():


    def __init__(self, config):

        self.config = config
        self.padding_idx = config.pad_token_id
        self.vocab_size = config.vocab_size

        self.embed_tokens = np.load('tinyllama/model.embed_tokens.weight_weights.npy', mmap_mode='r')

        self.layers = [DecoderLayer(config, layer_idx) for layer_idx in range(config.num_hidden_layers)]

        self.final_layernorm = RMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self._use_flash_attention_2 = False

        self.gradient_checkpointing = False
        # Initialize weights and apply final processing


    def get_input_embeddings(self):
        return self.embed_tokens

    def set_input_embeddings(self, value):
        self.embed_tokens = value


    def __call__(
        self,
        input_ids = None,
        attention_mask = None,
        position_ids = None,
        past_key_values = None,
        inputs_embeds= None,
        use_cache= None,
        output_attentions = None,
        output_hidden_states= None,
        return_dict = None,
    ) :
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache

        return_dict = False

        # retrieve input_ids and inputs_embeds
        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            batch_size, seq_length = input_ids.shape[:2]
        elif inputs_embeds is not None:
            batch_size, seq_length = inputs_embeds.shape[:2]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        past_key_values_length = 0



        if use_cache:
            use_legacy_cache = not isinstance(past_key_values, Cache)
            if use_legacy_cache:
                past_key_values = DynamicCache.from_legacy_cache(past_key_values)
            past_key_values_length = past_key_values.get_usable_length(seq_length)

        if position_ids is None:

            position_ids = np.arange(
                past_key_values_length, seq_length + past_key_values_length, dtype=np.int64
            )
            position_ids = np.expand_dims(position_ids, 0)

        if inputs_embeds is None:
            inputs_embeds = self.embed_tokens[input_ids]


        batch_size, seq_length = inputs_embeds.shape[:2]

        attention_mask = None
        hidden_states = inputs_embeds

        # decoder layers
        all_hidden_states = () if output_hidden_states else None
        all_self_attns = () if output_attentions else None
        next_decoder_cache = None

        for decoder_layer in self.layers:
            if output_hidden_states:
                all_hidden_states += (hidden_states,)

            layer_outputs = decoder_layer(
                hidden_states,
                attention_mask=attention_mask,
                position_ids=position_ids,
                past_key_value=past_key_values,
                output_attentions=output_attentions,
                use_cache=use_cache,
            )

            hidden_states = layer_outputs[0]

            if use_cache:
                next_decoder_cache = layer_outputs[2 if output_attentions else 1]

            if output_attentions:
                all_self_attns += (layer_outputs[1],)

        hidden_states = self.final_layernorm(hidden_states)

        # add hidden states from the last decoder layer
        if output_hidden_states:
            all_hidden_states += (hidden_states,)

        next_cache = None
        if use_cache:
            next_cache = next_decoder_cache.to_legacy_cache() if use_legacy_cache else next_decoder_cache
        last_hidden_state=hidden_states
        past_key_values=next_cache
        hidden_states=all_hidden_states
        attentions=all_self_attns
        return (
            last_hidden_state,
            past_key_values,
            hidden_states,
            attentions
        )


In [36]:
class ForCausalLM():


    # Copied from transformers.models.llama.modeling_llama.LlamaForCausalLM.__init__ with Llama->Phi,bias=False->bias=True
    def __init__(self, config):
        self.config = config

        self.model = Model(config)
        self.vocab_size = config.vocab_size
        self.lm_head = Linear_np(config.hidden_size, config.vocab_size, bias=True)

        weights =np.load(f'tinyllama/lm_head.weight_weights.npy', mmap_mode='r')



        self.lm_head.load(weights, bias=None)



    # Copied from transformers.models.llama.modeling_llama.LlamaForCausalLM.get_input_embeddings
    def get_input_embeddings(self):
        return self.model.embed_tokens

    # Copied from transformers.models.llama.modeling_llama.LlamaForCausalLM.set_input_embeddings
    def set_input_embeddings(self, value):
        self.model.embed_tokens = value

    # Copied from transformers.models.llama.modeling_llama.LlamaForCausalLM.get_output_embeddings
    def get_output_embeddings(self):
        return self.lm_head

    # Copied from transformers.models.llama.modeling_llama.LlamaForCausalLM.set_output_embeddings
    def set_output_embeddings(self, new_embeddings):
        self.lm_head = new_embeddings

    # Copied from transformers.models.llama.modeling_llama.LlamaForCausalLM.set_decoder
    def set_decoder(self, decoder):
        self.model = decoder

    # Copied from transformers.models.llama.modeling_llama.LlamaForCausalLM.get_decoder
    def get_decoder(self):
        return self.model


    def __call__(
        self,
        input_ids = None,
        attention_mask = None,
        position_ids = None,
        past_key_values = None,
        inputs_embeds = None,
        labels = None,
        use_cache = None,
        output_attentions = None,
        output_hidden_states= None,
        return_dict = None,
    ) :

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = False #return_dict if return_dict is not None else self.config.use_return_dict

        # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_states = outputs[0]
        logits = self.lm_head(hidden_states)
        #logits = softmax(logits, axis=-1)
        logits = logits

        loss = None

        logits=logits
        past_key_values=outputs[1]
        hidden_states=outputs[2]
        attentions=outputs[3]

        return (
            loss,
            logits,
            past_key_values,
            hidden_states,
            attentions,
        )

    # Copied from transformers.models.llama.modeling_llama.LlamaForCausalLM.prepare_inputs_for_generation
    def prepare_inputs_for_generation(
        self, input_ids, past_key_values=None, attention_mask=None, inputs_embeds=None, **kwargs
    ):
        if past_key_values is not None:

            if isinstance(past_key_values, Cache):
                cache_length = past_key_values.get_seq_length()
                past_length = past_key_values.seen_tokens
                max_cache_length = past_key_values.get_max_length()
            else:
                cache_length = past_length = past_key_values[0][0].shape[2]
                max_cache_length = None

            # Keep only the unprocessed tokens:
            # 1 - If the length of the attention_mask exceeds the length of input_ids, then we are in a setting where
            # some of the inputs are exclusively passed as part of the cache (e.g. when passing input_embeds as
            # input)
            if attention_mask is not None and attention_mask.shape[1] > input_ids.shape[1]:
                input_ids = input_ids[:, -(attention_mask.shape[1] - past_length) :]
            # 2 - If the past_length is smaller than input_ids', then input_ids holds all input tokens. We can discard
            # input_ids based on the past_length.
            elif past_length < input_ids.shape[1]:
                input_ids = input_ids[:, past_length:]
            # 3 - Otherwise (past_length >= input_ids.shape[1]), let's assume input_ids only has unprocessed tokens.

            # If we are about to go beyond the maximum cache length, we need to crop the input attention mask.
            if (
                max_cache_length is not None
                and attention_mask is not None
                and cache_length + input_ids.shape[1] > max_cache_length
            ):
                attention_mask = attention_mask[:, -max_cache_length:]

        position_ids = kwargs.get("position_ids", None)
        if attention_mask is not None and position_ids is None:
            # create position_ids on the fly for batch generation
            position_ids = attention_mask.long().cumsum(-1) - 1
            position_ids.masked_fill_(attention_mask == 0, 1)
            if past_key_values:
                position_ids = position_ids[:, -input_ids.shape[1] :]

        # if `inputs_embeds` are passed, we only want to use them in the 1st generation step
        if inputs_embeds is not None and past_key_values is None:
            model_inputs = {"inputs_embeds": inputs_embeds}
        else:
            model_inputs = {"input_ids": input_ids}

        model_inputs.update(
            {
                "position_ids": position_ids,
                "past_key_values": past_key_values,
                "use_cache": kwargs.get("use_cache"),
                "attention_mask": attention_mask,
            }
        )
        return model_inputs

In [37]:
model = ForCausalLM(config1)

In [38]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")


In [39]:
import random

def sample(probabilities, n):
    probabilities = softmax(probabilities)
    try:
    # Sample index from probabilities, they must sum to 1
        r = random.random()
        probabilities = probabilities
        cdf = 0.0
        for i in range(n):
            cdf += probabilities[i]
            if r < cdf:
                return i
        return n - 1  # In case of rounding errors
    except:
        print(probabilities)

# Example usage
probabilities = [0.1, 0.2, 0.3, 0.4]  # Example list of probabilities
n = len(probabilities)  # Number of probabilities
index = sample(probabilities, n)
print(index)


0


In [40]:
prompt = 'once upon a time there was little boy'
prompt= '''
CONTEXT
The Great Wall of China is a series of fortifications made of stone, brick, tamped earth, wood, and other materials, generally built along an east-to-west line across the historical northern borders of China to protect the Chinese states and empires against the raids and invasions of the various nomadic groups of the Eurasian Steppe.

The primary purpose of the Great Wall of China is
'''
prompt = '</s> \n<|user|>: hi \n<|assistant|>:'

inp = tokenizer.encode(prompt, add_special_tokens=False )
inps = model.prepare_inputs_for_generation(
        inp, past_key_values=None, attention_mask=None, inputs_embeds=None,
    )

input_ids = tokenizer.encode(prompt)

# Generate text token by token

for _ in range(5):
    outputs = model(np.array([input_ids]))
    predictions = outputs[1]

    # Get the predicted next token (you could apply more complex strategies here)
    next_token_id = np.argmax(predictions[:, -1, :], axis=-1)



    input_ids.append(next_token_id[0])
    print(tokenizer.decode(input_ids, skip_special_tokens=True))

    # Check if the generation reached an end-of-sequence token
    if next_token_id.item() == tokenizer.eos_token_id:
        break




 
<|user|>: hi 
<|assistant|>: Sure
 
<|user|>: hi 
<|assistant|>: Sure,
 
<|user|>: hi 
<|assistant|>: Sure, here
 
<|user|>: hi 
<|assistant|>: Sure, here'
 
<|user|>: hi 
<|assistant|>: Sure, here's


In [137]:
torch.argmax(predictions[:, -1, :], axis=-1, keepdim=True)

tensor([[2278]])

In [139]:
np.argmax(predictions[:, -1, :].numpy(), axis=-1)

array([2278])

In [133]:
input_ids.numpy()

array([[    1,  2748,  2501,   263,   931,   727,   471,  2217,  8023,
         4257,  5457, 29889,  5457,   471,   263,  9796,   322,  2562,
         9021,  2278]])

In [131]:
tokenizer.decode(input_ids[0], skip_special_tokens=True)

'once upon a time there was little boy named Jack. Jack was a happy and carefree child'

In [41]:
def generate(prompt, max_tokens=20, streamer=None):
    inp = tokenizer.encode(prompt, add_special_tokens=False )
    inps = model.prepare_inputs_for_generation(
            inp, past_key_values=None, attention_mask=None, inputs_embeds=None,
        )
    past_key_values = None
    res = prompt
    print(prompt, end="")
    input_ids = tokenizer.encode(prompt)

    # Generate text token by token

    for _ in range(max_tokens):
        outputs = model(np.array([input_ids]), use_cache=False)
        predictions = outputs[1]

        # Get the predicted next token (you could apply more complex strategies here)
        next_token_id = np.argmax(predictions[:, -1, :], axis=-1)



        input_ids.append(next_token_id[0])
        print(tokenizer.decode(input_ids, skip_special_tokens=True), end="")

        # Check if the generation reached an end-of-sequence token
        if next_token_id.item() == tokenizer.eos_token_id:
            break


In [42]:
generate('</s> \n<|user|>: hi \n<|assistant|>:', max_tokens=300)


</s> 
<|user|>: hi 
<|assistant|>: 
<|user|>: hi 
<|assistant|>: Sure 
<|user|>: hi 
<|assistant|>: Sure, 
<|user|>: hi 
<|assistant|>: Sure, here 
<|user|>: hi 
<|assistant|>: Sure, here'

KeyboardInterrupt: 

In [98]:
prompt = 'once upon a time there'

In [99]:
inp = tokenizer.encode(prompt, add_special_tokens=False )
inps = model.prepare_inputs_for_generation(
            np.array([inp]),
        )


In [100]:
ou1 = model(inps['input_ids'], use_cache=False)[1]
ou = model1(torch.tensor(inps['input_ids']), use_cache=False)[0]

In [125]:
torch.isclose(ou[-1], torch.Tensor(ou1[-1]), rtol=1e-2)

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [94]:
ou[-1][-1].shape

torch.Size([4, 32000])

In [95]:
ou1[-1][-1].shape

(32000,)

In [219]:
ou1.shape

(1, 4, 32000)

In [172]:
ou[-1][-1].detach().numpy()
np.argmax(ou[-1][-1].detach().numpy())

29892

In [175]:
val = sample(ou[-1][-1].detach().numpy(), vocab_size)


prompt = tokenizer.decode(val, skip_special_tokens=True)
res += prompt
print(prompt, end="")

,

In [81]:
from transformers import AutoModelForCausalLM
model1 = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [32]:
attn = Attention(config1, 0)
x = np.random.randn(1,1,2048)
inp = torch.Tensor(x)
attn(x)[0]

array([[[-0.2355665 , -0.01450975,  0.18568064, ...,  0.00186091,
         -0.14888893, -0.00527345]]])

In [33]:
inp = torch.Tensor(x)

In [34]:
model1.model.layers[0].self_attn(hidden_states=inp)

(tensor([[[-0.2356, -0.0145,  0.1857,  ...,  0.0019, -0.1489, -0.0053]]],
        grad_fn=<UnsafeViewBackward0>),
 None,
 None)

In [35]:
config1.num_hidden_layers

22

In [228]:
x = np.random.randn(1,1,2048).astype('f')
inp = torch.Tensor(x)

a = model.lm_head(x)



In [233]:
b = model1.lm_head(inp)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [230]:
np.argmax(a)

23641

In [232]:
np.argmax(b.detach().numpy())

23641

In [222]:
x = np.random.randn(1,1,2048).astype('f')
x.dtype

dtype('float32')

In [ ]:
attn = Attention(config1, 0)
x = np.random.randn(1,1,2048).astype('f')

inp = torch.Tensor(x)



for i in range(22):
  attn = model.model.layers[i].self_attn
  x = np.random.randn(1,1,2048).astype('f')

  inp = torch.Tensor(x)
  og = attn(x)[0]
  ot = model1.model.layers[i].self_attn(hidden_states=inp)[0]

  print(i,   torch.isclose(torch.Tensor(og), ot, rtol=1e-3 ))


In [42]:
attns = []
for i in range(22):
  attn = Attention(config1, i)
  attns.append(attn)

In [ ]:




for i in range(22):
  attn = attns[i]
  x = np.random.randn(1,1,2048).astype('f')

  inp = torch.Tensor(x)
  og = attn(x)[0]
  ot = model1.model.layers[i].self_attn(hidden_states=inp)[0]

  print(i,   torch.isclose(torch.Tensor(og), ot, rtol=1e-3 ))


In [44]:
decoders = []
for i in range(config1.num_hidden_layers):
  dec = DecoderLayer(config1, i)
  decoders.append(dec)


In [ ]:



for i in range(22):
  dec = decoders[i]
  x = np.random.randn(1,1,2048).astype('f')

  inp = torch.Tensor(x)
  og = dec(x)[0]
  ot = model1.model.layers[i](hidden_states=inp)[0]

  print(i,   torch.isclose(torch.Tensor(og), ot, rtol=1e-3 ))


In [107]:
for i in range(2):
  inp = model1.model.layers[i](inp)[0]
inp

tensor([[[ 0.5771,  0.4140, -0.6160,  ...,  1.6984, -1.4095, -0.0606]]],
       grad_fn=<AddBackward0>)

In [60]:
model1.model.layers[0](hidden_states=inp)

(tensor([[[-0.4942, -0.7665, -0.0165,  ...,  0.5256, -0.2423,  0.0855]]],
        grad_fn=<AddBackward0>),)

In [123]:
inp = tokenizer.encode('once upon a time', add_special_tokens=False )
inp = tokenizer.encode('once upon a time', add_special_tokens=False )
inps = model.prepare_inputs_for_generation(
            inp, past_key_values=None, attention_mask=None, inputs_embeds=None,
        )

In [325]:
inps

{'input_ids': array([[2748, 2501]]),
 'position_ids': None,
 'past_key_values': None,
 'use_cache': None,
 'attention_mask': None}

In [234]:
ou1 = model1.model(inps['input_ids'],)[1]

AttributeError: 'numpy.ndarray' object has no attribute 'device'

In [244]:
inps['input_ids'].shape

(1, 4)

In [278]:
inp = tokenizer.encode('once upon', add_special_tokens=False )
inps = model.prepare_inputs_for_generation(
            np.array([inp]),
        )


In [279]:
inps

{'input_ids': array([[2748, 2501]]),
 'position_ids': None,
 'past_key_values': None,
 'use_cache': None,
 'attention_mask': None}

In [295]:
x = model.model.get_input_embeddings()[inps['input_ids']]

In [296]:
x.shape

(1, 2, 2048)

In [294]:
model1.model.get_input_embeddings()(torch.tensor(inps['input_ids']))

tensor([[[-0.0175, -0.0198, -0.0063,  ..., -0.0039,  0.0028,  0.0035],
         [ 0.0019,  0.0018, -0.0049,  ...,  0.0093, -0.0027,  0.0070]]],
       grad_fn=<EmbeddingBackward0>)

In [304]:
inps

{'input_ids': array([[2748, 2501]]),
 'position_ids': None,
 'past_key_values': None,
 'use_cache': None,
 'attention_mask': None}

In [324]:
position_ids = torch.arange(
                0, 2, dtype=torch.long
            )
position_ids = position_ids.unsqueeze(0)
position_ids

tensor([[0, 1]])

In [322]:
inps['input_ids'].shape[:2]

(1, 2)

In [357]:
ou1 = model.model(inps['input_ids'],)[0]
ou = model1.model(torch.tensor(inps['input_ids'])).last_hidden_state

In [409]:
mask = torch.zeros(1,8)

In [410]:
ou = model1.model(torch.tensor(inps['input_ids']), attention_mask=mask).last_hidden_state

In [433]:
m =  np.triu(np.ones((1, 8)) * -1e9, k=1)
m.shape

(1, 8)

In [469]:
import numpy as np

def create_masked_context_mask(mask, diagonal=0, dtype=np.float32):
    """
    Apply a context mask to the given mask array.

    :param mask: The original mask array (2D or 4D).
    :param diagonal: The diagonal offset for the upper triangular part.
    :param dtype: The data type to determine the minimum value.
    :return: The masked array with applied context mask.
    """

    # Create an upper triangular matrix (context mask)
    context_mask = 1 - np.tril(np.ones_like(mask, dtype=int), k=0)


    # Apply the context mask to the original mask
    masked_mask = np.where(context_mask, np.finfo(dtype).min, mask)

    return masked_mask

# Example usage
mask = np.random.rand(4, 4)  # Replace with your actual mask
diagonal = 0
masked_mask = create_masked_context_mask(mask, diagonal)
masked_mask

array([[ 5.07360245e-01, -3.40282347e+38, -3.40282347e+38,
        -3.40282347e+38],
       [ 7.78231403e-01,  5.85227053e-01, -3.40282347e+38,
        -3.40282347e+38],
       [ 9.13163643e-01,  7.81205803e-01,  3.25738826e-01,
        -3.40282347e+38],
       [ 1.32595720e-01,  6.71988259e-01,  6.83150214e-02,
         5.01991165e-01]])

In [468]:
np.tril(np.ones_like(mask, dtype=int), k=0)


array([[1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 1, 0],
       [1, 1, 1, 1]])

In [453]:
torch.tril(torch.ones(4, 4))

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])

In [448]:
model1.model(torch.tensor(inps['input_ids']), attention_mask=torch.tensor(modified_mask)).last_hidden_state

tensor([[[-0.0112,  0.1805, -0.0019,  ..., -0.2434,  0.0460,  0.0762],
         [ 0.0561,  0.1115,  0.0196,  ..., -0.5309,  0.1469,  0.0058],
         [ 0.0305,  0.0511,  0.0558,  ..., -0.2500,  0.0426, -0.0475],
         ...,
         [ 0.0514,  0.1340, -0.0147,  ..., -0.2421,  0.0670,  0.0383],
         [ 0.0742,  0.0647, -0.0473,  ..., -0.4106,  0.0404, -0.0191],
         [ 0.0521,  0.1862,  0.0205,  ..., -0.4150,  0.0413,  0.0671]]],
       grad_fn=<AddBackward0>)

In [449]:
model1.model(torch.tensor(inps['input_ids'])).last_hidden_state

tensor([[[-1.1165e-02,  1.8055e-01, -1.8866e-03,  ..., -2.4341e-01,
           4.5975e-02,  7.6172e-02],
         [ 3.7167e-01,  8.5975e-02,  4.4363e-01,  ..., -2.5829e+00,
           3.8793e-01, -2.0166e-01],
         [ 1.7158e+00,  8.1196e-02, -2.3189e-02,  ..., -1.8756e+00,
          -5.0874e-01,  1.1068e+00],
         ...,
         [ 2.9602e-01, -9.0776e-01, -2.9161e-01,  ..., -5.0614e+00,
           3.5623e-01, -4.4828e-01],
         [-2.2318e-01, -5.3769e-01, -1.4666e-01,  ..., -5.4657e-01,
           3.1356e-01, -1.3549e-01],
         [-3.9934e-02,  1.8208e-02, -1.5282e-01,  ..., -4.5241e+00,
           1.3718e-01, -3.5677e-01]]], grad_fn=<AddBackward0>)

In [412]:
model1.model(torch.tensor(inps['input_ids'])).last_hidden_state

tensor([[[-1.1165e-02,  1.8055e-01, -1.8866e-03,  ..., -2.4341e-01,
           4.5975e-02,  7.6172e-02],
         [ 3.7167e-01,  8.5975e-02,  4.4363e-01,  ..., -2.5829e+00,
           3.8793e-01, -2.0166e-01],
         [ 1.7158e+00,  8.1196e-02, -2.3189e-02,  ..., -1.8756e+00,
          -5.0874e-01,  1.1068e+00],
         ...,
         [ 2.9602e-01, -9.0776e-01, -2.9161e-01,  ..., -5.0614e+00,
           3.5623e-01, -4.4828e-01],
         [-2.2318e-01, -5.3769e-01, -1.4666e-01,  ..., -5.4657e-01,
           3.1356e-01, -1.3549e-01],
         [-3.9934e-02,  1.8208e-02, -1.5282e-01,  ..., -4.5241e+00,
           1.3718e-01, -3.5677e-01]]], grad_fn=<AddBackward0>)

In [354]:
import torch.nn as nn
class LlamaRMSNorm1(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        LlamaRMSNorm is equivalent to T5LayerNorm
        """
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        return hidden_states


In [356]:
model1.model.norm = LlamaRMSNorm1(0)

In [358]:
ou1

array([[[-3.15384308, -1.24769725, -0.37631961, ...,  3.86259941,
          1.18668998, -0.153979  ],
        [-3.14749838, -1.33593831, -0.57573195, ...,  3.75710779,
          1.10185667, -0.38560823]]])

In [359]:
ou

tensor([[[-1.1164e-02,  1.8055e-01, -1.8863e-03,  ..., -2.4341e-01,
           4.5975e-02,  7.6172e-02],
         [ 3.7168e-01,  8.5976e-02,  4.4363e-01,  ..., -2.5829e+00,
           3.8793e-01, -2.0166e-01]]], grad_fn=<AddBackward0>)

In [99]:
ou1[-1][-1].shape

(32000,)

In [51]:
x = np.random.randn(1,1,2048)
y = x
inp = torch.Tensor(x)



In [307]:
import copy

In [347]:
x

array([[[-0.01745605, -0.01977539, -0.0062561 , ..., -0.00393677,
          0.00280762,  0.00346375],
        [ 0.0019455 ,  0.00180054, -0.00491333, ...,  0.00933838,
         -0.00265503,  0.00704956]]], dtype=float32)

In [348]:
hs = copy.deepcopy(x)
model.model(hs)

IndexError: arrays used as indices must be of integer (or boolean) type

In [393]:
hs = copy.deepcopy(x)
hs = torch.tensor(hs)
model1.model.layers[0](hs)

RuntimeError: expected m1 and m2 to have the same dtype, but got: double != float

In [328]:
for i in range(22):
  hs = model.model.layers[i](hs, use_cache=False)[0]

hs

array([[[-1.43325806, -0.59864963, -0.16895729, ...,  1.70678401,
          0.55277126, -0.06969217],
        [-1.42975007, -0.64070808, -0.25837512, ...,  1.65944496,
          0.51303097, -0.17445325]]])

In [329]:
model.model.final_layernorm(hs)

array([[[-3.15384308, -1.24769725, -0.37631961, ...,  3.86259941,
          1.18668998, -0.153979  ],
        [-3.14749838, -1.33593831, -0.57573195, ...,  3.75710779,
          1.10185667, -0.38560823]]])

In [330]:
hs = copy.deepcopy(x)
hs = torch.tensor(hs)
for i in range(22):
  hs = model1.model.layers[i](hs)[0]
hs

tensor([[[-1.4333, -0.5986, -0.1690,  ...,  1.7068,  0.5528, -0.0697],
         [-1.4297, -0.6407, -0.2584,  ...,  1.6594,  0.5130, -0.1745]]],
       grad_fn=<AddBackward0>)

In [331]:
model1.model.norm(hs)

tensor([[[-3.1538, -1.2477, -0.3763,  ...,  3.8626,  1.1867, -0.1540],
         [-3.1475, -1.3359, -0.5757,  ...,  3.7571,  1.1019, -0.3856]]],
       grad_fn=<MulBackward0>)

In [334]:
hs = copy.deepcopy(x)
hs = torch.tensor(hs)
batch_size, seq_length = hs.shape[:2]

In [337]:
position_ids = torch.arange(
                0, seq_length + 0, dtype=torch.long
            )
position_ids = position_ids.unsqueeze(0)

In [340]:
hs

tensor([[[-0.0175, -0.0198, -0.0063,  ..., -0.0039,  0.0028,  0.0035],
         [ 0.0019,  0.0018, -0.0049,  ...,  0.0093, -0.0027,  0.0070]]])

In [342]:
hidden_states = hs
for decoder_layer in model1.model.layers:

    layer_outputs = decoder_layer(
        hidden_states,
        position_ids=position_ids
    )

    hidden_states = layer_outputs[0]

In [343]:
hidden_states

tensor([[[-1.4333, -0.5986, -0.1690,  ...,  1.7068,  0.5528, -0.0697],
         [-1.4297, -0.6407, -0.2584,  ...,  1.6594,  0.5130, -0.1745]]],
       grad_fn=<AddBackward0>)

In [344]:
model1.model.norm(hidden_states)

tensor([[[-3.1538, -1.2477, -0.3763,  ...,  3.8626,  1.1867, -0.1540],
         [-3.1475, -1.3359, -0.5757,  ...,  3.7571,  1.1019, -0.3856]]],
       grad_fn=<MulBackward0>)

In [53]:

m = RMSNorm(2048, layer_idx=10)
m(x)

torch.isclose(torch.Tensor(m(x)), model1.model.layers[10].input_layernorm(inp), rtol=1e-3 )

tensor([[[True, True, True,  ..., True, True, True]]])

In [54]:
x = np.random.randn(1,1,2048)
y = x
inp = torch.Tensor(x)


print(decoders[0](x))
model1.model.layers[0](inp)[0]

(array([[[-0.94062658,  0.21811317,  0.38043982, ...,  1.18543976,
          2.22400013, -1.09295196]]]),)


tensor([[[-0.9406,  0.2181,  0.3804,  ...,  1.1854,  2.2240, -1.0930]]],
       grad_fn=<AddBackward0>)

In [55]:
torch.isclose(torch.Tensor(decoders[0](x)),model1.model.layers[0](inp)[0], rtol=1e-2 )

<ipython-input-55-86fc63ca5fd1>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  torch.isclose(torch.Tensor(decoders[0](x)),model1.model.layers[0](inp)[0], rtol=1e-2 )


tensor([[[[True, True, True,  ..., True, True, True]]]])